In [1]:
conda install -c conda-forge r-irkernel



Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "C:\Users\marti\anaconda3\Scripts\conda-script.py", line 11, in <module>
    from conda.cli import main
ModuleNotFoundError: No module named 'conda'


In [8]:
#install datatable 
install.packages("data.table")



  There is a binary version available but the source version is later:
           binary source needs_compilation
data.table 1.14.0 1.14.8              TRUE

  Binaries will be installed
package 'data.table' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\marti\AppData\Local\Temp\RtmpAreh6u\downloaded_packages


In [1]:
rm(list = ls()) # Borro todos los objetos
gc() # Garbage Collection

require("data.table")
require("rpart")
require("parallel")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,507947,27.2,1140321,60.9,621911,33.3
Vcells,937717,7.2,8388608,64.0,1602022,12.3


Loading required package: data.table
Warning message:
"package 'data.table' was built under R version 3.6.3"Loading required package: rpart
Loading required package: parallel


In [2]:
# esqueleto de grid search
# se espera que los alumnos completen lo que falta
#   para recorrer TODOS cuatro los hiperparametros


PARAM <- list()
# reemplazar por las propias semillas
semillas <- c(100019,100043,100049,100057,100069)

#------------------------------------------------------------------------------
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)
#   crea una particion 70, 30

particionar <- function(data, division, agrupa = "", campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}
#------------------------------------------------------------------------------

ArbolEstimarGanancia <- function(semilla, param_basicos) {
  # particiono estratificadamente el dataset
  particionar(dataset, division = c(7, 3), agrupa = "clase_ternaria", seed = semilla)

  # genero el modelo
  # quiero predecir clase_ternaria a partir del resto
  modelo <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_basicos
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion <- predict(modelo, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad

  # prediccion es una matriz con TRES columnas,
  #  llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  # cada columna es el vector de probabilidades


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test <- dataset[
    fold == 2,
    sum(ifelse(prediccion[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada <- ganancia_test / 0.3

  return(ganancia_test_normalizada)
}
#------------------------------------------------------------------------------

ArbolesMontecarlo <- function(semillas, param_basicos) {
  # la funcion mcmapply  llama a la funcion ArbolEstimarGanancia
  #  tantas veces como valores tenga el vector  ksemillas
  ganancias <- mcmapply(ArbolEstimarGanancia,
    semillas, # paso el vector de semillas
    MoreArgs = list(param_basicos), # aqui paso el segundo parametro
    SIMPLIFY = FALSE,
    mc.cores = 1
  ) # se puede subir a 5 si posee Linux o Mac OS

  ganancia_promedio <- mean(unlist(ganancias))

  return(ganancia_promedio)
}
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

# Aqui se debe poner la carpeta de la computadora local
#setwd("~/buckets/b1/") # Establezco el Working Directory
# cargo los datos


In [3]:
file_path <- normalizePath("C:/Users/marti/oneDrive/Documentos/Data Science Maestria/laboratorio/labo2023ba/datasets_dataset_pequeno.csv")
dataset <- fread(file_path)

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]


# genero el archivo para Kaggle
# creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning
dir.create("./exp/", showWarnings = FALSE)
dir.create("./exp/HT2020/", showWarnings = FALSE)
archivo_salida <- "./exp/HT2020/gridsearch.txt"

# Escribo los titulos al archivo donde van a quedar los resultados
# atencion que si ya existe el archivo, esta instruccion LO SOBREESCRIBE,
#  y lo que estaba antes se pierde
# la forma que no suceda lo anterior es con append=TRUE
cat(
  file = archivo_salida,
  sep = "",
  "cp", "\t",
  "max_depth", "\t",
  "min_split", "\t",
  "min_bucket", "\t",
  "ganancia_promedio", "\n"
)


In [4]:
#dataset.head(10)
head(dataset,10)

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
31116053,202107,1,0,0,50,201,13073.36,161035.99,3163.52,...,10,0.00,-82110.00,0.00,6060,121585.39,75,0,77617.41,CONTINUA
31116803,202107,1,0,0,59,326,2640.48,56751.51,5232.22,...,3,0.00,-85347.94,17.59,4211,21596.40,7,0,4938.33,CONTINUA
31117730,202107,1,0,0,68,272,1343.51,20201.31,582.94,...,24,4692.00,-4692.00,0.00,7951,1529.02,2,0,1407.60,CONTINUA
31117908,202107,1,0,0,80,326,5523.19,55827.28,2712.26,...,3,0.00,-26420.00,0.00,7702,12815.99,8,0,856.29,CONTINUA
31117977,202107,1,0,0,60,261,4819.35,37386.40,1878.40,...,24,174063.72,-160642.13,12.89,7949,157186.12,27,0,9548.22,CONTINUA
31120119,202107,1,0,0,71,379,1374.50,55964.94,1561.94,...,10,0.00,-10424.07,0.00,8508,9831.19,2,0,656.88,CONTINUA
31121017,202107,1,0,0,67,326,2579.35,18260.09,1958.62,...,3,0.00,-4048.02,0.00,8739,3988.20,3,0,211.14,CONTINUA
31121264,202107,1,0,0,46,311,-1451.10,-5036.02,2348.34,...,10,30480.40,-140760.00,0.00,8480,62938.18,49,0,14193.30,CONTINUA
31121718,202107,1,0,0,66,326,37560.90,478240.90,2827.01,...,10,3873.57,-3075.19,0.00,8416,3761.81,1,0,258.06,CONTINUA
31122951,202107,1,0,0,59,270,4119.31,60639.40,1959.26,...,3,0.00,-78615.19,0.00,4629,92695.38,29,0,9876.66,CONTINUA


: 

In [ ]:
for (vmax_depth in c(4, 6, 8, 10)) {
  for (vmin_split in c(1000, 800, 600, 400, 200, 100)) {
    for (cp in c(-0.3, -0.5, -0.7)) {
      for (minbucket in c(5, 3, 1,300,500)) {
        # notar como se agrega
        
        # vminsplit  minima cantidad de registros en un nodo para hacer el split
        param_basicos <- list(
          "cp" = cp, # complejidad minima
          "maxdepth" = vmax_depth,
          "minsplit" = vmin_split,
          "minbucket" = minbucket # minima cantidad de registros en una hoja
        )
        
        # profundidad máxima del arbol
        # Un solo llamado, con la semilla 17
        ganancia_promedio <- ArbolesMontecarlo(semillas, param_basicos)
        
        # escribo los resultados al archivo de salida
        cat(
          file = archivo_salida,
          append = TRUE,
          sep = "",
          cp, "\t",
          vmax_depth, "\t",
          vmin_split, "\t",
          minbucket, "\t",
          ganancia_promedio, "\n"
        )
      }
    }
  }
}


: 

: 